<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
!sqlite3 movies.sqlite < lab2.sql

In [2]:
%load_ext sql

And then we import our movie database

In [3]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [4]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

## SQL statements to review at the lab session

Write SQL code for the following tasks:

+ Show the names of all movies.

In [5]:
%%sql
SELECT *
FROM movies

 * sqlite:///movies.sqlite
Done.


imdb_id,title,year,time
dc1234567,Dösjebro Calling,2010,180
ts0000000,Trainspotting,1980,167
nn1234567,Någon film,2000,80


+ Show the performance dates for one of the movies.

In [6]:
%%sql
SELECT DISTINCT date
FROM performances
WHERE imdb_id in (
    SELECT imdb_id
    FROM movies
    WHERE title = 'Dösjebro Calling'
)

 * sqlite:///movies.sqlite
Done.


date
2022-02-08
2022-02-09


+ Show all data concerning performances at a given theatere
  on a given date.

In [7]:
%%sql
SELECT *
FROM performances
WHERE name = 'Filmstaden Eslöv' AND date = '2022-02-08'

 * sqlite:///movies.sqlite
Done.


performance_id,name,imdb_id,date,time
ca1e0be72bd4e8843a506aae50b4ccea,Filmstaden Eslöv,nn1234567,2022-02-08,12:00
9289d529d275cc39ac2b1fb91e7e82ca,Filmstaden Eslöv,nn1234567,2022-02-08,14:00
241ff33e35a0db87e40c3e3e70cc86a6,Filmstaden Eslöv,nn1234567,2022-02-08,16:00
2e354dd36a28c892f039fbaecf65f01d,Filmstaden Eslöv,nn1234567,2022-02-08,18:00


+ List all customers

In [8]:
%%sql
SELECT *
FROM customers

 * sqlite:///movies.sqlite
Done.


username,name,password
abc,Göran Persson,lösenord1
def,Klas Petter,lösenord2
ghi,Tove Styrke,lösenord3


+ List all tickets

In [9]:
%%sql
SELECT *
FROM tickets

 * sqlite:///movies.sqlite
Done.


ticket_id,username,performance_id


## Things to try out yourselves (not reviewed at the lab session)

During lab 3 you'll write a REST service for the database
you just created, and for the server to work properly, the
things below should work.

If you have any questions about the material below, feel
free to ask them at the QA sessions (see Moodle for how to
sign up for them).


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [10]:
%%sql
INSERT 
INTO tickets
VALUES ((lower(hex(randomblob(16)))), 'abc', 'ca1e0be72bd4e8843a506aae50b4ccea')


 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [11]:
%%sql
SELECT ticket_id
FROM tickets
WHERE rowid = last_insert_rowid();

 * sqlite:///movies.sqlite
Done.


ticket_id
58decb965bbc56c49f6841e64ebac235


+ Try to insert two movie theaters with the same name (this
  should fail).

In [12]:
%%sql
INSERT 
INTO theatres
VALUES ('Filmstaden Lund', 50)

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) UNIQUE constraint failed: theatres.name
[SQL: INSERT 
INTO theatres
VALUES ('Filmstaden Lund', 50)]
(Background on this error at: http://sqlalche.me/e/gkpj)


+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [13]:
%%sql
INSERT
INTO performances
VALUES (lower(hex(randomblob(16))), 'Filmstaden Helsingborg', 'dc1234567', '2022-02-08', '18:30')

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT
INTO performances
VALUES (lower(hex(randomblob(16))), 'Filmstaden Helsingborg', 'dc1234567', '2022-02-08', '18:30')]
(Background on this error at: http://sqlalche.me/e/gkpj)


+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [15]:
%%sql
INSERT 
INTO tickets
VALUES ((lower(hex(randomblob(16)))), 'abc', 'ae0da58f56850bedc53b1b2f57801d32')

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT 
INTO tickets
VALUES ((lower(hex(randomblob(16)))), 'abc', 'ae0da58f56850bedc53b1b2f57801d32')]
(Background on this error at: http://sqlalche.me/e/gkpj)
